In [1]:
sc

In [2]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
import time
import sys
from generator_images import captcha_generator as gen
import matplotlib.pyplot as plt

In [3]:
chars = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabdefghijlmnqrtuwxy"
width, height, n_len, n_class = 140, 44, 6, len(chars)    

In [4]:
sqlContext.sql("DROP TABLE images6")

DataFrame[]

In [5]:
sqlContext.sql("CREATE EXTERNAL TABLE images6 (id INT COMMENT 'id of images', mat ARRAY<ARRAY<ARRAY<ARRAY<BIGINT>>>> \
COMMENT 'array of list') COMMENT \
'This is used to store images' ROW FORMAT DELIMITED \
FIELDS TERMINATED BY '\t' STORED AS TEXTFILE LOCATION 'hdfs://master:9000/user/hive/images6'")
sqlContext.sql("DESCRIBE TABLE images6").show()

+--------+--------------------+-------------+
|col_name|           data_type|      comment|
+--------+--------------------+-------------+
|      id|                 int| id of images|
|     mat|array<array<array...|array of list|
+--------+--------------------+-------------+



In [6]:
num = 0

In [7]:
start_time = time.time()
image = gen(width=width, height=height)
while num < 10:
    X, y = image.next()
    newimage = sqlContext.createDataFrame([(num, X.tolist())], ['id', 'mat'])
    newimage.write.mode('append').insertInto('images6')
    num += 1
    end_time = time.time()
    print '--------------------------%d----------------------------'%num
    print 'the total time is', end_time - start_time
    print 'the average time is', (end_time - start_time)/64.0 /num

--------------------------1----------------------------
the total time is 21.8697168827
the average time is 0.341714326292
--------------------------2----------------------------
the total time is 28.4441800117
the average time is 0.222220156342
--------------------------3----------------------------
the total time is 34.832764864
the average time is 0.181420650333
--------------------------4----------------------------
the total time is 42.0784738064
the average time is 0.164369038306
--------------------------5----------------------------
the total time is 48.8968849182
the average time is 0.152802765369
--------------------------6----------------------------
the total time is 54.9693570137
the average time is 0.143149367223
--------------------------7----------------------------
the total time is 62.1490428448
the average time is 0.138725542064
--------------------------8----------------------------
the total time is 69.8048458099
the average time is 0.136337589473
-----------------

In [8]:
sqlContext.sql("DESCRIBE TABLE images6").show()

+--------+--------------------+-------------+
|col_name|           data_type|      comment|
+--------+--------------------+-------------+
|      id|                 int| id of images|
|     mat|array<array<array...|array of list|
+--------+--------------------+-------------+



In [9]:
sqlContext.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|  images5|      false|
| default|  images6|      false|
+--------+---------+-----------+



In [10]:
dataset = sqlContext.sql("SELECT * FROM images6")

In [11]:
type(dataset)

pyspark.sql.dataframe.DataFrame

In [12]:
dataset.repartition(14)

DataFrame[id: int, mat: array<array<array<array<bigint>>>>]

In [13]:
newdataset = dataset.repartition(14)

In [43]:
def work_train(worker_id, iterator):
    print 'worker_id', worker_id
    print 'iterator', iterator
    yield (worker_id, sum(iterator))

In [15]:
newdataset.rdd.mapPartitionsWithIndex(work_train)

PythonRDD[99] at RDD at PythonRDD.scala:48

In [16]:
type(newdataset.rdd.mapPartitionsWithIndex(work_train))

pyspark.rdd.PipelinedRDD

In [17]:
pipeRdd = newdataset.rdd.mapPartitionsWithIndex(work_train)

In [18]:
pipeRdd.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 14.0 failed 4 times, most recent failure: Lost task 9.3 in stage 14.0 (TID 1725, master, executor 5): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/ubuntu/Download/spark-2.1.1/python/pyspark/worker.py", line 174, in main
    process()
  File "/home/ubuntu/Download/spark-2.1.1/python/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/ubuntu/Download/spark-2.1.1/python/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-14-5f16c48d7018>", line 2, in work_train
TypeError: unsupported operand type(s) for +: 'int' and 'Row'

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spar

In [19]:
x = sc.parallelize(['A','A','B'])

In [21]:
type(x)

pyspark.rdd.RDD

In [24]:
x.collect()

['A', 'A', 'B']

In [30]:
df = x.map(lambda x:(x, )).toDF()

In [31]:
df.show()

+---+
| _1|
+---+
|  A|
|  A|
|  B|
+---+



In [26]:
myFloatRdd = sc.parallelize([1.0,2.0,3.0])

In [28]:
df = myFloatRdd.map(lambda x: (x, )).toDF()

In [29]:
df.show()

+---+
| _1|
+---+
|1.0|
|2.0|
|3.0|
+---+



In [32]:
from pyspark.sql import Row

In [33]:
row = Row('myval')

In [39]:
df1 = myFloatRdd.map(row).toDF()

In [42]:
df1.show()

+-----+
|myval|
+-----+
|  1.0|
|  2.0|
|  3.0|
+-----+



In [58]:
df1.rdd.mapPartitionsWithIndex(work_train_new).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 60.0 failed 4 times, most recent failure: Lost task 7.3 in stage 60.0 (TID 2051, master, executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/ubuntu/Download/spark-2.1.1/python/pyspark/worker.py", line 174, in main
    process()
  File "/home/ubuntu/Download/spark-2.1.1/python/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/ubuntu/Download/spark-2.1.1/python/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-57-e63d65d0d0e9>", line 4, in work_train_new
AttributeError: 'itertools.chain' object has no attribute 'asDict'

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.ap

In [46]:
x.collect()

['A', 'A', 'B']

In [47]:
myFloatRdd.collect()

[1.0, 2.0, 3.0]

In [57]:
def work_train_new(worker_id, iterator):
    print 'worker_id', worker_id
    print 'iterator', iterator
    yield (worker_id, sum(iterator.asDict().values()))

In [48]:
myFloatRdd.mapPartitionsWithIndex(work_train).collect()

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 1.0),
 (5, 0),
 (6, 0),
 (7, 0),
 (8, 0),
 (9, 2.0),
 (10, 0),
 (11, 0),
 (12, 0),
 (13, 3.0)]

In [49]:
df1.rdd.collect()

[Row(myval=1.0), Row(myval=2.0), Row(myval=3.0)]

In [50]:
r0 = df1.rdd.collect()[0]

In [51]:
r0

Row(myval=1.0)

In [56]:
r0.asDict().values()

[1.0]